In [1]:
import numpy as np
import pandas as pd

In [2]:
# 2017.4.10. drug-drug similarity

In [3]:
str_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_structure.tsv")

In [4]:
gold_se_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/5_gold_drug_drug_sim_predict.tsv")

In [5]:
gold_se_sim_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/5_2_gold_drug_drug_sim_mat_predict.tsv", index_col=0)

In [6]:
drugbank = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/drug_set.txt", header=None)
drugbank = drugbank.rename(columns = {0:'DrugBank_ID', 1:'Drug_Name', 2:'InChI', 3:'SMILES', 4:'State'})
drugbank.head(1)

,DrugBank_ID,Drug_Name,InChI,SMILES,State
0,DB00006,Bivalirudin,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,approved


In [28]:
gold_drug_id = gold_se_sim_mat.columns.values.tolist()
print len(gold_drug_id)

446


In [29]:
str_dr_id = str_sim['DrugBank_ID1'].drop_duplicates()
print len(str_dr_id)

456


In [15]:
# 기존에 구해놓은 similarity score가 커버할 수 있는 현재 gold_drug_id
fi_drug_id = str_dr_id[str_dr_id.isin(gold_drug_id)]
print len(fi_drug_id)

427


In [31]:
# drugbank에서 smiles를 제공하는 gold drug id
db_dr_id = drugbank[drugbank['DrugBank_ID'].isin(gold_drug_id)]['DrugBank_ID']
print len(db_dr_id)

#기존 similarity score가 커버하지 못하는 drug들 --> similarity score를 추가로 구해야함.
fi_drug_id2 = db_dr_id[~(db_dr_id.isin(fi_drug_id))]
print len(fi_drug_id2)

434
7


In [54]:
# 기존의 drug sim에서 gold drug를 filter한것.
print len(str_sim)
temp = str_sim[(str_sim['DrugBank_ID1'].isin(fi_drug_id))]
print len(temp)
fi_drug_str_sim = temp[temp['DrugBank_ID2'].isin(fi_drug_id)]
print len(fi_drug_str_sim)

207936
194712
182329


In [56]:
import pybel

fi_drug = fi_drug_id.tolist() + fi_drug_id2.tolist() # db_dr_id랑 같음..
print len(fi_drug)

fi_drug_str_sim2 = fi_drug_str_sim
for i in fi_drug_id2:
    smi1 = drugbank[drugbank.DrugBank_ID == i]['SMILES'].tolist()[0]
    for j in fi_drug:
        smi2 = drugbank[drugbank.DrugBank_ID == j]['SMILES'].tolist()[0]
        
        smiles = [smi1, smi2]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        
        temp_df = pd.DataFrame([[i, j, sim]], columns = ['DrugBank_ID1','DrugBank_ID2', 'Similarity_Score'])
        fi_drug_str_sim2 = fi_drug_str_sim2.append(temp_df)

for i in fi_drug:
    smi1 = drugbank[drugbank.DrugBank_ID == i]['SMILES'].tolist()[0]
    for j in fi_drug_id2:
        smi2 = drugbank[drugbank.DrugBank_ID == j]['SMILES'].tolist()[0]
        
        smiles = [smi1, smi2]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        
        temp_df = pd.DataFrame([[i, j, sim]], columns = ['DrugBank_ID1','DrugBank_ID2', 'Similarity_Score'])
        fi_drug_str_sim2 = fi_drug_str_sim2.append(temp_df)

434


In [57]:
print len(fi_drug_str_sim2)
print len(fi_drug_str_sim2.drop_duplicates())

188405
188356


In [59]:
# 기존 side effect similarity에서 gold에 해당하는것 filter
temp = gold_se_sim_mat.filter(items = db_dr_id)
fi_drug_se_sim = temp.filter(items = db_dr_id, axis=0)
print len(fi_drug_se_sim)
fi_drug_se_sim.head(1)

434


,DB00114,DB00126,DB00136,DB00153,DB00162,DB00165,DB00175,DB00176,DB00177,DB00178,...,DB01576,DB01577,DB01590,DB01623,DB02546,DB03904,DB04272,DB04844,DB04861,DB09026
DB00114,1.0,0.932755,0.900217,0.922993,0.933839,0.743492,0.886117,0.856833,0.901302,0.873102,...,0.89154,0.92679,0.87744,0.907809,0.874187,0.907267,0.927332,0.880694,0.87039,0.887744


In [62]:
db_dr_id.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_gold_drug.tsv", sep='\t', index=False)
db_dr_id.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/1_gold_drug.tsv", sep='\t', index=False)
fi_drug_se_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_se_sim_mat.tsv",sep='\t')
fi_drug_se_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_se_sim_mat.tsv",sep='\t')
fi_drug_str_sim2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_str_sim.tsv", sep='\t', index=False)
fi_drug_str_sim2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_str_sim.tsv", sep='\t', index=False)

In [68]:
fi_drug_str_sim_mat = pd.DataFrame(index = db_dr_id.tolist(),columns=db_dr_id.tolist())
fi_drug_str_sim_mat.fillna(0.0)

for index, row in fi_drug_str_sim2.iterrows():
    fi_drug_str_sim_mat[row['DrugBank_ID2']][row['DrugBank_ID1']] = row['Similarity_Score']
print len(fi_drug_str_sim_mat)
fi_drug_str_sim_mat.head(1)

434


,DB00114,DB00126,DB00136,DB00153,DB00162,DB00165,DB00175,DB00176,DB00177,DB00178,...,DB01576,DB01577,DB01590,DB01623,DB02546,DB03904,DB04272,DB04844,DB04861,DB09026
DB00114,1,0.0872483,0.0305344,0.0307692,0.0406504,0.646465,0.075,0.15894,0.16568,0.127907,...,0.0892857,0.0862069,0.0852018,0.0913706,0.111111,0.0102041,0.0619469,0.146067,0.123529,0.156977


In [69]:
fi_drug_str_sim_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_str_sim_mat.tsv",sep='\t')
fi_drug_str_sim_mat.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_str_sim_mat.tsv",sep='\t')

In [ ]:
# gold_association

In [70]:
gold_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/PREDICT_data/2_2_gold_learning_asso.tsv")

In [72]:
print len(gold_asso)
temp = gold_asso[(gold_asso['drug_id'].isin(db_dr_id))]
print len(temp)
pos = temp[temp.association ==1]
fi_disease_id = pos['disease_id'].drop_duplicates()
fi_learning_asso = temp[temp['disease_id'].isin(fi_disease_id)]
print len(fi_learning_asso)

103146
100361
99493


In [73]:
a = fi_learning_asso[fi_learning_asso.association ==1]
b = fi_learning_asso[fi_learning_asso.association ==0]
print len(a)
print len(b)
print len(a['drug_id'].drop_duplicates())
print len(b['drug_id'].drop_duplicates())
print len(a['disease_id'].drop_duplicates())
print len(b['disease_id'].drop_duplicates())

1330
98163
434
434
233
233


In [74]:
fi_learning_asso.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/1_gold_learning_asso.tsv", sep='\t', index=False)
fi_learning_asso.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/new_predict/1_gold_learning_asso.tsv", sep='\t', index=False)

In [ ]:
#disease disease similarity

In [75]:
di_feature = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/8_gold_standard_disease_binary_matrix.tsv", index_col=0)

In [76]:
di_feature.head(1)

,102300,102400,102500,103780,104130,104300,106400,107650,109100,109200,...,607631,607681,607682,607850,608088,608105,608217,608320,608583,608622
100188812,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
str_fi_disease_id = list()
for i in fi_disease_id:
    str_fi_disease_id.append(str(i))

In [78]:
di_feature.columns.values.tolist()[0]

'102300'

In [79]:
str_fi_disease_id[0]

'103780'

In [80]:
a = di_feature.filter(items = str_fi_disease_id)
a.head(1)

,103780,608622,161900,145590,147530,157300,608088,140600,147480,600351,...,173200,115195,256700,601608,194070,115300,277350,240400,206200,250790
100188812,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
di_ge_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_genetic.tsv")
di_phe_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_mimminerc.tsv", index_col=0)

In [123]:
print len(di_ge_sim)
temp = di_ge_sim[(di_ge_sim['Disease_ID1'].isin(fi_disease_id))]
print len(temp)
fi_disease_ge_sim = temp[(temp['Disease_ID2'].isin(fi_disease_id))]
print len(fi_disease_ge_sim)

55225
54755
54289


In [121]:
print len(di_phe_sim)
temp = di_phe_sim.filter(items = str_fi_disease_id)
fi_di_phe_sim = temp.filter(items = fi_disease_id, axis=0)
print len(fi_di_phe_sim)
fi_di_phe_sim.head(1)

235
233


,103780,608622,161900,145590,147530,157300,608088,140600,147480,600351,...,173200,115195,256700,601608,194070,115300,277350,240400,206200,250790
103780,1.0,0.231225,0.112214,0.106257,0.078062,0.127741,0.108825,0.0,0.152292,0.057957,...,0.031407,0.150525,0.136234,0.16048,0.12331,0.178678,0.141056,0.200361,0.162371,0.187959


In [130]:
fi_disease_id.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_gold_disease_id.tsv", sep='\t', index=False)
fi_disease_id.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/new_predict/1_gold_disease_id.tsv", sep='\t', index=False)
fi_di_phe_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_phe_sim_mat.tsv",sep='\t')
fi_di_phe_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_phe_sim_mat.tsv",sep='\t')
fi_disease_ge_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_ge_sim.tsv", sep='\t', index=False)
fi_disease_ge_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/new_predict/1_diesease_disease_ger_sim.tsv", sep='\t', index=False)

In [182]:
whole_disease = fi_disease_ge_sim
whole_disease['PheSim'] = -2.0
for index, row in fi_disease_ge_sim.iterrows():
    ids = row.tolist()
    di1 = ids[0]
    di2 = ids[1]
    sim = fi_di_phe_sim.ix[di2][str(int(di1))]
    whole_disease.set_value(index, 'PheSim', sim)
    cnt = cnt +1
    if cnt == 2:
        break
whole_disease = whole_disease.rename(columns={'Similarity_Score':'GeSim'})
whole_disease.head(1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Disease_ID1,Disease_ID2,GeSim,PheSim
0,102300,102300,1.0,1.0


In [189]:
whole_disease.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_whole_sim.tsv", sep='\t', index=False)
whole_disease.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_whole_sim.tsv", sep='\t', index=False)

In [216]:
fi_disease_ge_sim_mat = pd.DataFrame(index = fi_disease_id.tolist(),columns=fi_disease_id.tolist())
fi_disease_ge_sim_mat.fillna(0.0)

for index, row in fi_disease_ge_sim.iterrows():
    fi_disease_ge_sim_mat[row['Disease_ID2']][row['Disease_ID1']] = row['Similarity_Score']

In [218]:
fi_disease_ge_sim_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_ge_sim_mat.tsv",sep='\t')
fi_disease_ge_sim_mat.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_ge_sim_mat.tsv",sep='\t')